# Fine Tuning with Unsloth and SFTTrainer (Supervised Finetuning Trainer)

In [ ]:
# !pip uninstall xformers

Found existing installation: xformers 0.0.27.post2
Uninstalling xformers-0.0.27.post2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/xformers-0.0.27.post2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/xformers/*
Proceed (Y/n)? y
  Successfully uninstalled xformers-0.0.27.post2


In [ ]:
!pip install xformers

  Using cached xformers-0.0.27.post2-cp310-cp310-manylinux2014_x86_64.whl.metadata (1.0 kB)
Using cached xformers-0.0.27.post2-cp310-cp310-manylinux2014_x86_64.whl (20.8 MB)


In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
from torch import __version__; from packaging.version import Version as V
# xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
# !pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
!pip install --no-deps trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # Use 4bit quantization to reduce memory usage.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Optimized
    bias = "none",    # Optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
user_prompt="""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    # instructions = examples["instructions"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = user_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("saruo06/train-llama3-lawscribe", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

# Interactive Chat Tasks

In [ ]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    user_prompt.format(
        # "Paraphrase sentence", # instruction
        "What is a loan agreement?", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

# NLP Tasks

In [ ]:
inputs = tokenizer(
[
    user_prompt.format(
        # "Paraphrase sentence", # instruction
        "Paraphrase the sentence - country india, city bangalore. the rules are ask me before painting, pay 50000 fine for any minor damages anf 2 lak for major damages, no renting out to other people, the mentioned party should definitely be stayimg in that house. provide me these rules in proper legal terms to include in lease agreement", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

# Summarization Tasks

In [ ]:
sample_text = """
LOAN AGREEMENT BETWEEN

Aryan Majhi
AND
Anupam Kumar

THIS AGREEMENT made and entered into at Patna this 29th day of August, 2024, BETWEEN Aryan Majhi, hereinafter called "the Lender," AND Anupam Kumar, hereinafter called "the Borrower," and reference to the parties hereto shall mean and include their respective heirs, executors, administrators, and assigns;

WHEREAS the Borrower is in need of funds and hence has approached the Lender to grant him an interest-free loan of Rs. 7500/- (Rupees Seven Thousand Five Hundred Only) for a period of 10 years;

AND WHEREAS the Lender has agreed to grant a loan to the Borrower, free of interest, as the Lender and the Borrower have known each other for several years;
AND WHEREAS the parties hereto are desirous of recording the terms and conditions of this loan in writing;
NOW THIS AGREEMENT WITNESSETH and it is hereby agreed by and between the parties hereto as under:

    1. The Borrower hereto, being in need of money, has requested the Lender to give him an interest-free loan of Rs. 7500/- (Rupees Seven Thousand Five Hundred Only) to enable him to purchase a residential flat, to which the Lender has agreed.

    2. The said loan is required by the Borrower for a period of 10 years, commencing from 29/08/2024 and terminating on 29/08/2034.

    3. The Borrower hereby agrees and undertakes to return the loan of Rs. 7500/- (Rupees Seven Thousand Five Hundred Only), in instalments, within the aforesaid period of 10 years and gives his personal guarantee for the same.

    4. The terms and conditions of this Agreement are arrived at by the mutual consent of the parties hereto.

IN WITNESS WHEREOF the parties hereto have hereunto set and subscribed their respective hands the day and year first hereinabove written.

SIGNED AND DELIVERED by the Within-named Lender in the presence of: Rishita Shah
SIGNED AND DELIVERED by the Within-named Borrower in the presence of: AXB

Give me a summary of the document within 200 words
"""

In [ ]:
inputs = tokenizer(
[
    user_prompt.format(
        sample_text, # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
#Saving model
import torch

torch.save(model.state_dict(), 'lawscribe_model.pth')